## Example A — Frustum (no longitudinal slope)
Given:
- Outer polygon area A_o = 1200 m²
- Inner polygon area A_i = 400 m²
- Depth D = 3.0 m

We compute the frustum volume as:

V = (D / 3) * (A_o + A_i + sqrt(A_o * A_i))

In [ ]:
import math
A_o = 1200.0  # m^2
A_i = 400.0   # m^2
D = 3.0       # m
V_frustum = (D / 3.0) * (A_o + A_i + math.sqrt(A_o * A_i))
V_frustum

The computed frustum volume above is the excavation volume (m³) for the basin with the provided outer and inner plan areas and a uniform depth. This method assumes the inner polygon was generated by offset = depth / side_slope using the upstream depth.

## Example B — Frustum with longitudinal slope (Simpson integration)
Same A_o and A_i, upstream depth D_up = 3.0 m, longitudinal_slope = 20% over flow_length = 50 m.
Compute upstream, midpoint and downstream frustum volumes and use Simpson's rule to integrate along the flow path.

In [ ]:
A_o = 1200.0
A_i = 400.0
D_up = 3.0
longitudinal_slope_pct = 20.0  # percent
flow_length = 50.0  # m
# Downstream depth (linear variation)
D_down = D_up + (longitudinal_slope_pct / 100.0) * flow_length
D_mid = (D_up + D_down) / 2.0
def frustum(D, Ao, Ai):
    import math
    return (D / 3.0) * (Ao + Ai + math.sqrt(Ao * Ai))
V_up = frustum(D_up, A_o, A_i)
V_mid = frustum(D_mid, A_o, A_i)
V_down = frustum(D_down, A_o, A_i)
V_simpson = (V_up + 4.0 * V_mid + V_down) / 6.0
D_up, D_mid, D_down, V_up, V_mid, V_down, V_simpson

Interpretation:
- `D_up`, `D_mid`, `D_down` show the depth at the three sampled locations.
- Simpson's formula approximates the integral of the frustum volume along the flow path; results are in m³.
- If downstream depth is negative (rare), the implementation clamps it to 0 before applying the frustum formula.

## Example C — DEM differencing (raster cells)
Given a 1 m × 1 m DEM grid and a polygon covering 10 cells with positive differences (original - modified) listed below, compute the excavation volume by summing positive differences times cell area.

In [ ]:
diffs = [0.5, 0.2, 1.0, 0.0, 0.6, 0.0, 0.8, 0.0, 0.3, 0.4]  # meters (orig - mod)
cell_area = 1.0  # m^2 (1m x 1m cells)
volume_dem = sum(d for d in diffs if d > 0) * cell_area
volume_dem

Notes:
- Real DEM differencing loops over raster cell centers and checks polygon inclusion.
- Cells with `nodata` in either DEM are skipped.
- For non-square pixels, `cell_area` should be computed from the raster transform: `abs(transform.a * transform.e)`.

## Example D — Cross-section area integration (cut vs fill)
This worked example uses a small, symmetric set of offsets and shows how cut/fill polygons are constructed and integrated.
Offsets are [-2, -1, 0, 1, 2] m with uniform spacing 1 m.
Existing elevations: [100, 101, 102, 101, 100]
Final elevations: [100, 100.5, 102.5, 100.5, 100]
We compute cut (where final < existing) and fill (where final > existing) areas using segment-wise averages.

In [ ]:
offsets = [-2, -1, 0, 1, 2]

existing = [100, 101, 102, 101, 100]


Explanation:
- Each segment between offsets contributes `(avg_final - avg_existing) * width` to cut (if negative) or fill (if positive).
- The app uses more samples (e.g., 201 offsets) to capture template geometry smoothly.

---
### Next steps
- You can copy and run these cells locally to reproduce the numeric examples.
- For DEM differencing on a real DEM, use `rasterio` to read arrays and `shapely` to test point-in-polygon for cell centers.
- For basin examples using your DEM and polygon, paste your polygon coordinates and a small DEM tile and we can walk through a concrete case.